# Using images in a survey
This notebook provides sample code for using images with an [EDSL](https://docs.expectedparrot.com/) survey.

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Scenarios
A `Scenario` is a dictionary containing a key/value pair that is used to add data or content to questions in an EDSL survey.
Scenarios allow you create variations and versions of questions efficiently, and with data or content from different sources.

EDSL provides a variety of methods for automatically generating scenarios from PDFs, CSVs, docs, tables, lists, dicts -- and images.
In the steps below we demonstrate how to create a scenario for an image and use it in a survey.

[Learn more about working with scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html).

### Creating a scenario
We start by creating a `Scenario` for an image.
For purposes of demonstration, we post a PNG image to the Coop, and then retrieve it as a scenario (this can be done by [any user with a Coop account](https://www.expectedparrot.com/login)).

Code for posting a PNG file to the Coop (uncomment and replace with your own file):

In [1]:
from edsl.scenarios.FileStore import PNGFileStore
fs = PNGFileStore("parrot_logo.png")
info = fs.push()
print(info)

{'description': 'File: parrot_logo.png', 'object_type': 'scenario', 'url': 'https://www.expectedparrot.com/content/b9632b87-41a4-4a34-ab1e-bd18add8f4a7', 'uuid': 'b9632b87-41a4-4a34-ab1e-bd18add8f4a7', 'version': '0.1.39.dev1', 'visibility': 'unlisted'}


Retrieving a Coop file to use as a scenario (replace with the UUID of a desired object):

In [2]:
from edsl.scenarios.FileStore import PNGFileStore

In [3]:
png_file = PNGFileStore.pull(info["uuid"])

Converting the retrieved file into a scenario:

In [4]:
from edsl import Scenario

In [5]:
s = Scenario.from_image(png_file.to_tempfile(), "parrot_logo")

Alternative method for creating a scenario from a local file:

In [6]:
# s = Scenario.from_image("parrot_logo.png", "parrot_logo")

Verify the scenario keys to use in questions:

In [7]:
s.keys()

['parrot_logo']

## Creating questions using the image
Next we construct questions with the image scenario.
Note that we use a `{{ placeholder }}` for the scenario key.
We also demonstrate how to pipe an answer into a follow-on question:

In [8]:
from edsl import QuestionYesNo, QuestionFreeText, QuestionList, Survey

In [9]:
q1 = QuestionYesNo(
    question_name = "animal",
    question_text = "Is there an animal in this image? {{ parrot_logo }}"
)

In [10]:
q2 = QuestionFreeText(
    question_name = "identify",
    question_text = "Identify the animal in this image: {{ parrot_logo }}"
)

In [11]:
q3 = QuestionList(
    question_name = "colors",
    question_text = "What color(s) is the animal?", 
)

In [12]:
survey = Survey(questions = [q1, q2, q3])

Adding logic and rules for adminitering the questions:

In [13]:
survey = (
    survey
    .add_stop_rule(q1, "animal == 'No'")
    .add_targeted_memory(q3, q2)
)

We use the scenario by adding it to the survey when we run it, as with any other scenarios:

In [14]:
results = survey.by(s).run()

In [15]:
results.select("animal", "identify", "colors", "colors_comment")

answer.animal,answer.identify,answer.colors,comment.colors_comment
Yes,The animal in the image is a parrot.,"['green', 'red', 'blue', 'yellow']","# Parrots are often brightly colored and can have a combination of green, red, blue, and yellow feathers."


In [16]:
from edsl import Notebook

In [17]:
n = Notebook(path = "image_scenario_example.ipynb")

In [18]:
n.push(description = "Using an image scenario", visibility = "public")

{'description': 'Using an image scenario',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/b72f3990-0630-4aa7-99c8-2230d51376d1',
 'uuid': 'b72f3990-0630-4aa7-99c8-2230d51376d1',
 'version': '0.1.39.dev1',
 'visibility': 'public'}